# Training of our ML Model

**SageMaker Studio Kernel**: Data Science

In this exercise you will do:
 - Run a Preprocessing Job using Amazon SageMaker Processing Job
 - Run a Tensorflow Training Job using Amazon SageMaker Training Job
 - Register a new version of the trained model in the Amazon SageMaker Model Registry

***

## Part 1/2 - Setup
Here we'll import some libraries and define some variables. You can also take a look on the scripts that were previously created for preparing the data and training our model.

In [ ]:
import boto3
import logging
import sagemaker
from sagemaker.tensorflow import TensorFlow

In [ ]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger(__name__)

In [ ]:
sagemaker_client = boto3.client("sagemaker")
s3_client = boto3.client("s3")

***

### Global configurations

Configuration variables used for Processing, Training, and registration

In [ ]:
region = boto3.session.Session().region_name
role_name = "mlops-sagemaker-execution-role"
role = "arn:aws:iam::{}:role/{}".format(boto3.client('sts').get_caller_identity().get('Account'), role_name)

kms_account_id = boto3.client('sts').get_caller_identity().get('Account')

kms_alias = "ml-kms"

bucket_name = ""

In [ ]:
boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client("sagemaker")
runtime_client = boto_session.client("sagemaker-runtime")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_runtime_client=runtime_client,
    default_bucket=bucket_name
)

In [ ]:
kms_key = "arn:aws:kms:{}:{}:alias/{}".format(region, kms_account_id, kms_alias)

***

## Part 2/2: Run the end to end ML workflow

### Step 1/2: Create the Training Job

#### Compress source code for installing additional python modules

In [ ]:
!pygmentize ./../algorithms/training/src/train.py

In [ ]:
! ./../algorithms/buildspec.sh training $bucket_name

#### Define input variables

In [ ]:
processing_output_files_path = "data/output"

training_artifact_path = "artifact/training"
training_artifact_name = "sourcedir.tar.gz"
training_output_files_path = "models"
training_framework_version = "2.4"
training_python_version = "py37"
training_instance_count = 1
training_instance_type = "ml.p2.xlarge"
training_hyperparameters = {
    "epochs": 5,
    "learning_rate": 1.45e-4,
    "batch_size": 100
}

#### Create Estimator

Lets start a training job using a Tensorflow Estimator

In [ ]:
estimator = TensorFlow(
    entry_point="train.py",
    framework_version=training_framework_version,
    py_version=training_python_version,
    source_dir="s3://{}/{}/{}".format(bucket_name,
                                      training_artifact_path,
                                      training_artifact_name
                                      ),
    output_path="s3://{}/{}".format(bucket_name,
                                    training_output_files_path),
    hyperparameters=training_hyperparameters,
    enable_sagemaker_metrics=True,
    metric_definitions=[
        {
            'Name': 'Test accuracy',
            'Regex': 'Test accuracy:.* ([0-9\\.]+)'
        }
    ],
    role=role,
    instance_count=training_instance_count,
    instance_type=training_instance_type,
    output_kms_key=kms_key,
    disable_profiler=True
)

In [ ]:
estimator.fit(
    inputs={
        "train": "s3://{}/{}/train".format(
            bucket_name,
            processing_output_files_path
        ),
        "test": "s3://{}/{}/test".format(
            bucket_name,
            processing_output_files_path
        )
    },
    logs="Rules"
)

***

### Step 2/2: Register Model in the Model Registry

#### Input Parameters

In [ ]:
inference_instance_type = "ml.m5.xlarge"

model_package_group_name = "ml-end-to-end-group"
model_approval_status = "PendingManualApproval"

In [ ]:
estimator.register(
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=[inference_instance_type],
    transform_instances=[inference_instance_type]
)

We have just seen how to process, train, and version ML models by using Amazon SageMaker Jobs. Now we are ready to execute our end to end workflow using an Amazon SageMaker Pipeline

 > [SageMaker-Pipeline](./06-SageMaker-Pipeline-Training.ipynb)

If we want to test the execution of a Custom Script container, we can execute the next lab
 > [Train-Custom-Script-Container](./04-Train-Build-Model-Custom-Script-Container.ipynb)

If we want to run Amazon SageMaker Hyperparameter optimization jobs, for identifying the best set of hyperparameters for the model, we can execute the lab
 > [Hyperparameter-Optimization](./05-Hyperparameter-Optimization.ipynb)